In [1]:
import sys
sys.path.append(r"C:\\Users\\a-sochat\\development\\pysweep2.0")

import numpy as np
import pandas as pd

import pysweep
from pysweep import sweep, Measurement

import qcodes 

In [2]:
class NoisyResistor(qcodes.Instrument): 
    def __init__(self, name, resistance):
        super().__init__(name)
    
        self._resistance = resistance
        self._bias = 0
    
        self.add_parameter(
            name="bias", 
            set_cmd=self._set_bias,
            unit="V"
        )
        
        self.add_parameter(
            name="current", 
            get_cmd=self._get_current, 
            unit="A"
        )
        
        self.connect_message()
        
    def _set_bias(self, bias): 
        self._bias = bias
    
    def _get_current(self): 
        return self._bias / self._resistance + np.random.normal(0.0, 0.1)
    
    def connect_message(self, idn_param='IDN', begin_time=None): 
        
        con_msg = ('Connected to: noisy resistor')
        print(con_msg)
        return {}

r = NoisyResistor("R", 10)

Connected to: noisy resistor


In [3]:
station = qcodes.Station(r)

In [4]:
Measurement.use_storage("spyview")
Measurement.set_station(station)

In [5]:
setup = lambda s, n: {}
cleanup = lambda s, n: {}
run = lambda s, n, v: {"run": {"unit": "#", "value": v, "independent_parameter": True}}

In [6]:
run_n_times = lambda count: sweep(run, np.arange(count))

In [7]:
measurement = Measurement(
    setup, 
    cleanup, 
    run_n_times(100)(
        sweep(r.bias, np.linspace(0, 10, 5))(
            r.current
        )
    )
)

In [8]:
out = measurement.run(store_parameters=["R_current"])

[ True False False False False  True False False False False  True False
 False False False  True False False False False  True False False False
 False  True False False False False  True False False False False  True
 False False False False  True False False False False  True False False
 False False  True False False False False  True False False False False
  True False False False False  True False False False False  True False
 False False False  True False False False False  True False False False
 False  True False False False False  True False False False False  True
 False False False False  True False False False False  True False False
 False False  True False False False False  True False False False False
  True False False False False  True False False False False  True False
 False False False  True False False False False  True False False False
 False  True False False False False  True False False False False  True
 False False False False  True False False False Fa

In [9]:
out._output_file_path

'C:\\Users\\a-sochat\\data\\2017-12-19\\2017-12-19_005.dat'

In [137]:
def insert_field(array, position, field_name, field_dtype):
    
    names = page.dtype.names
    
    new_dtype = [array.dtype[n] for n in names]
    new_dtype.insert(position, field_dtype)
    
    new_names = list(names)
    new_names.insert(position, field_name)
    
    new_array = np.zeros(array.shape, dtype=list(zip(new_names, new_dtype)))
    
    for name in names: 
        new_array[name] = array[name]
    
    return new_array

In [138]:
new_page = insert_field(page, 1, "empty", np.dtype((int, (1, ))))

In [140]:
new_page["R_bias [V]"][:4]

array([[ 0. ],
       [ 2.5],
       [ 5. ],
       [ 7.5]])

In [83]:
names

('R_bias [V]', 'run [#]', 'R_current [A]')

In [122]:
new_dtype = [page.dtype[n] for n in page.dtype.names]

In [123]:
new_dtype.insert(1, np.dtype((int, (1,))))

In [124]:
new_dtype

[dtype(('<f8', (1,))),
 dtype(('<i4', (1,))),
 dtype(('<i4', (1,))),
 dtype(('<f8', (1,)))]

In [125]:
new_names = list(names)

In [126]:
new_names.insert(1, "empty")

In [127]:
new_names

['R_bias [V]', 'empty', 'run [#]', 'R_current [A]']

In [133]:
new_page = np.zeros(page.shape, dtype=list(zip(new_names, new_dtype)))

In [134]:
new_page.dtype

dtype([('R_bias [V]', '<f8', (1,)), ('empty', '<i4', (1,)), ('run [#]', '<i4', (1,)), ('R_current [A]', '<f8', (1,))])

In [135]:
new_page["R_bias [V]"] = page["R_bias [V]"]

In [136]:
new_page["empty"]

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [141]:
import qcodes

In [142]:
qcodes.data.location

<module 'qcodes.data.location' from 'C:\\Users\\a-sochat\\AppData\\Local\\Continuum\\Anaconda3\\envs\\pysweep\\lib\\site-packages\\qcodes\\data\\location.py'>